In [1]:
import torch
import gpytorch
import math
from matplotlib import cm
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [51]:
def franke(X, Y):
    term1 = .75*torch.exp(-((9*X - 2).pow(2) + (9*Y - 2).pow(2))/4)
    term2 = .75*torch.exp(-((9*X + 1).pow(2))/49 - (9*Y + 1)/10)
    term3 = .5*torch.exp(-((9*X - 7).pow(2) + (9*Y - 3).pow(2))/4)
    term4 = .2*torch.exp(-(9*X - 4).pow(2) - (9*Y - 7).pow(2))

    f = term1 + term2 + term3 - term4
    dfx = -2*(9*X - 2)*9/4 * term1 - 2*(9*X + 1)*9/49 * term2 + \
          -2*(9*X - 7)*9/4 * term3 + 2*(9*X - 4)*9 * term4
    dfy = -2*(9*Y - 2)*9/4 * term1 - 9/10 * term2 + \
          -2*(9*Y - 3)*9/4 * term3 + 2*(9*Y - 7)*9 * term4

    return f, dfx, dfy
xv, yv = torch.meshgrid([torch.linspace(0, 1, 10), torch.linspace(0, 1, 10)])
train_x = torch.cat((
    xv.contiguous().view(xv.numel(), 1),
    yv.contiguous().view(yv.numel(), 1)),
    dim=1
)

f, dfx, dfy = franke(train_x[:, 0], train_x[:, 1])
train_y = torch.stack([f, dfx, dfy], -1).squeeze(1)

train_y += 0.05 * torch.randn(train_y.size())

In [59]:
print(train_y.numel())
print(train_x.numel())

1000
2000


In [8]:
train_x = (np.loadtxt('train_x.csv', delimiter=',', skiprows=1))[:1000,:]
train_y = (np.loadtxt('train_y.csv', delimiter=',', skiprows=1))[:1000]
test_x = (np.loadtxt('test_x.csv', delimiter=',', skiprows=1))[:1000]
train_x = torch.Tensor.double(torch.from_numpy(train_x))
train_y = torch.Tensor.double(torch.from_numpy(train_y))
test_x = torch.Tensor.double(torch.from_numpy(test_x))

In [3]:
def get_kernel(kernel, composition="addition"):
    base_kernel = []
    if "RBF" in kernel:
        base_kernel.append(gpytorch.kernels.RBFKernel())
    if "linear" in kernel:
        base_kernel.append(gpytorch.kernels.LinearKernel())
    if "quadratic" in kernel:
        base_kernel.append(gpytorch.kernels.PolynomialKernel(power=2))
    if "Matern-1/2" in kernel:
        base_kernel.append(gpytorch.kernels.MaternKernel(nu=1/2))
    if "Matern-3/2" in kernel:
        base_kernel.append(gpytorch.kernels.MaternKernel(nu=3/2))
    if "Matern-5/2" in kernel:
        base_kernel.append(gpytorch.kernels.MaternKernel(nu=5/2))
    if "Cosine" in kernel:
        base_kernel.append(gpytorch.kernels.CosineKernel())

    if composition == "addition":
        base_kernel = gpytorch.kernels.AdditiveKernel(*base_kernel)
    elif composition == "product":
        base_kernel = gpytorch.kernels.ProductKernel(*base_kernel)
    else:
        raise NotImplementedError
    kernel = gpytorch.kernels.ScaleKernel(base_kernel)
    return kernel

In [11]:
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean() #mean model
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel()) #Kernel

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x) #multivariate gaussian

likelihood = gpytorch.likelihoods.MultitaskGaussianLikelihood(num_tasks=3)  # Value + x-derivative + y-derivative
model = ExactGP(train_x, train_y, likelihood).double()

In [12]:
import os
smoke_test = ('CI' in os.environ)
training_iter = 2 if smoke_test else 50


# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam([
    {'params': model.parameters()},  # Includes GaussianLikelihood parameters
], lr=0.05)

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

for i in range(training_iter):
    optimizer.zero_grad()
    output = model(train_x)
    #print(train_y.dim())
    loss = -mll(output, train_y)
    loss.backward()
    print("Iter %d/%d - Loss: %.3f   lengthscales: %.3f, %.3f   noise: %.3f" % (
        i + 1, training_iter, loss.item(),
        model.covar_module.base_kernel.lengthscale.squeeze()[0],
        model.covar_module.base_kernel.lengthscale.squeeze()[1],
        model.likelihood.noise.item()
    ))
    optimizer.step()

RuntimeError: [enforce fail at ..\c10\core\CPUAllocator.cpp:79] data. DefaultCPUAllocator: not enough memory: you tried to allocate 32000000 bytes.

In [27]:
print(loss.item())

1.2174568176269531


In [23]:
torch.manual_seed(0)
num_training = 50
kernels = ["RBF", "linear", "quadratic", "Matern-1/2", "Matern-3/2", "Matern-5/2", "Cosine"]
composition = ["addition", "product"]
#train_x = (torch.rand(num_training) - 0.5) * 10
#train_y = regression_function(train_x)
#test_x = torch.linspace(-6, 6, 100)
print(type(train_x))
linestyles = ['-', ':', '--', '-.', 
             (0, (1, 10)), (0, (5, 10)), (0, (3, 5, 1, 5)), (0, (3, 5, 1, 5, 1, 5))
             ]

best_kernel = {}
for i, kernel in enumerate(kernels):
    model = ExactGP(train_x, train_y, get_kernel(kernel))
    
    model.train()
    optimizer = torch.optim.Adam([{'params': model.parameters()}], lr=0.1)
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(model.likelihood, model)
    training_iter = 100
    #print(type(model.likelihood),type(model))
    losses = []
    lengthscale = []
    outputscale = []
    noise = []

    for _ in range(training_iter):
        # Zero gradients from previous iteration
        optimizer.zero_grad()
        # Output from model
        
        output = model(train_x)
        print(type(output),type(train_y))
        # Calc loss and backprop gradients
        loss = -mll(output, train_y)
        loss.backward()

        losses.append(loss.item())
        lengthscale.append(model.length_scale.item())
        outputscale.append(model.output_scale.item())
        noise.append(model.likelihood.noise.item())
        optimizer.step()
    plt.plot(losses, label=f"{kernel}", linestyle=linestyles[i], linewidth=4)
    best_kernel[kernel] = (loss.item(), model)
plt.legend(loc="best")
plt.xlabel("Num Iteration")
plt.ylabel("MLL Loss")

plt.show()

plt.figure()
best_model = min(best_kernel.values(), key=lambda x: x[0])[1]
plot_model(best_model, train_x, train_y, test_x)
test_y = regression_function(test_x, noise=0).detach()
plt.plot(test_x, test_y,'k-', label='Noise-free Function')
plt.legend(loc='upper left')
    
plt.show()

<class 'torch.Tensor'>
<class 'gpytorch.distributions.multitask_multivariate_normal.MultitaskMultivariateNormal'> <class 'torch.Tensor'>


AttributeError: 'ExactGP' object has no attribute 'length_scale'